In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
!pip install contractions
import contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 9.3 MB/s eta 0:00:00


In [3]:
!pip install evaluate
import evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.5 MB/s eta 0:00:00a 0:00:01


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
cannot import name 'PartialState' from 'accelerate.state' (/opt/conda/lib/python3.10/site-packages/accelerate/state.py)

In [4]:
import re
from matplotlib import pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    roc_auc_score,
    roc_curve,
    auc,
    confusion_matrix,
    classification_report,
)

### Uploading train data

In [5]:
train_tw = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')

## Exploratory data analysis

### Looking at the data

In [ ]:
train_tw.head()

In [ ]:
train_tw.keyword.value_counts().index.to_list()

In [ ]:
train_tw.info()

In [ ]:
locations = train_tw.location.value_counts().index.to_list()
print(locations)

In [ ]:
train_tw.shape

In [ ]:
train_tw.target.value_counts()

In [ ]:
# print (train_tw.loc[3107: 3142, 'text'])
for row in range(3107, 3142):
    print (train_tw.loc[row, 'text'])

#### Let us now see the distribution of the tweets lengths in quantiles.

In [ ]:
train_tw['tweet_length'] = [len(tweet.split()) for tweet in train_tw.text]
train_tw.head()

In [ ]:
train_tw.tweet_length.value_counts()

In [ ]:
train_tw.tweet_length.nunique()

In [ ]:
print(train_tw['tweet_length'].quantile([0.01, 0.05, 0.10, 0.5, 0.9, 0.95, 0.99]))

In [ ]:
def plot_tweets_length_distrib(column):

    fig, ax = plt.subplots(1, 3, figsize=(16, 4))
    fig = plt.figure(figsize=(5, 10))

    lengths = [len(tweet.split()) for tweet in train_tw[column]]
    ax[0].hist(lengths, bins=31, linewidth=0.5, edgecolor="white")

    lengths = [len(tweet.split()) for tweet in train_tw[train_tw['target'] == 1][column]]
    ax[1].hist(lengths, bins=31, linewidth=0.5, edgecolor="white")

    lengths = [len(tweet.split()) for tweet in train_tw[train_tw['target'] == 0][column]]
    ax[2].hist(lengths, bins=31, linewidth=0.5, edgecolor="white")

    names = ['Common distribution', 'Distribution of disaster tweets.',
             'Distribution of calm tweets.']
    for i in range(len(names)):
        ax[i].set_title(names[i])
        ax[i].set_xlabel('Length')
        ax[i].set_ylabel('Number of tweets')

In [ ]:
plot_tweets_length_distrib('text')

#### Processing tweets. Let us try to find in the tweets characteristic symbols, like as URL's, html's, special characters, @ etc. In case some of them are found, we remove them.

In [6]:
# We introduce counters for every matching found to see how many of them are found:
matches = {'URL': 0, 'html': 0, 'spec_char': 0, 'at': 0, 'hashtags': 0, 'non_ascii': 0,
           'multiples_!': 0, 'multiples_?': 0, 'typos': 0, 'acronyms': 0, 'abbr': 0}
        
# Patterns for URL's
https_pattern = re.compile(r"https?://\S+|www\.\S+")
# Patterns for html's
html_pattern = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
# Patterns for special characters
emoji_pattern = re.compile(
                            '['
                            u'\U0001F600-\U0001F64F'  # emoticons
                            u'\U0001F300-\U0001F5FF'  # symbols & pictographs
                            u'\U0001F680-\U0001F6FF'  # transport & map symbols
                            u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
                            u'\U00002702-\U000027B0'
                            u'\U000024C2-\U0001F251'
                            ']+',
                            flags=re.UNICODE
    )
# Patterns for character sequences with At symbol (@)
at_pattern = re.compile(r'@\S*')
# Patterns for hashtag symbols
hashtag_pattern = re.compile(r'#([^\s]+)')
# Patterns for non_ascii characters:
non_ascii_pattern = re.compile(r'[^\x00-\x7f]')
# Patterns for sequences of several '!'
multiples_pattern_1 = re.compile(r'!+')
# Patterns for sequences of several '?'
multiples_pattern_2 = re.compile(r'\?+')

# Taking into account typos, slang and other:
sample_typos_slang = {
                        "w/e": "whatever",
                        "usagov": "usa government",
                        "recentlu": "recently",
                        "ph0tos": "photos",
                        "amirite": "am i right",
                        "exp0sed": "exposed",
                        "<3": "love",
                        "luv": "love",
                        "amageddon": "armageddon",
                        "trfc": "traffic",
                        "16yr": "16 year"
                        }
sample_typos_slang_pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) \
                            for key in sample_typos_slang.keys()) + r')(?!\w)')

# Acronyms
sample_acronyms =  { 
                    "mh370": "malaysia airlines flight 370",
                    "okwx": "oklahoma city weather",
                    "arwx": "arkansas weather",    
                    "gawx": "georgia weather",  
                    "scwx": "south carolina weather",  
                    "cawx": "california weather",
                    "tnwx": "tennessee weather",
                    "azwx": "arizona weather",  
                    "alwx": "alabama weather",
                    "usnwsgov": "united states national weather service",
                    "2mw": "tomorrow"
                    }
sample_acronyms_pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) \
                            for key in sample_acronyms.keys()) + r')(?!\w)')

# Some common english abbreviations: 
sample_abbr = {     "$" : " dollar ",
                    "€" : " euro ",
                    "4ao" : "for adults only",
                    "a.m" : "before midday",
                    "a3" : "anytime anywhere anyplace",
                    "aamof" : "as a matter of fact",
                    "acct" : "account",
                    "adih" : "another day in hell",
                    "afaic" : "as far as i am concerned",
                    "afaict" : "as far as i can tell",
                    "afaik" : "as far as i know",
                    "afair" : "as far as i remember",
                    "afk" : "away from keyboard",
                    "app" : "application",
                    "approx" : "approximately",
                    "apps" : "applications",
                    "asap" : "as soon as possible",
                    "asl" : "age, sex, location",
                    "atk" : "at the keyboard",
                    "ave." : "avenue",
                    "aymm" : "are you my mother",
                    "ayor" : "at your own risk", 
                    "b&b" : "bed and breakfast",
                    "b+b" : "bed and breakfast",
                    "b.c" : "before christ",
                    "b2b" : "business to business",
                    "b2c" : "business to customer",
                    "b4" : "before",
                    "b4n" : "bye for now",
                    "b@u" : "back at you",
                    "bae" : "before anyone else",
                    "bak" : "back at keyboard",
                    "bbbg" : "bye bye be good",
                    "bbc" : "british broadcasting corporation",
                    "bbias" : "be back in a second",
                    "bbl" : "be back later",
                    "bbs" : "be back soon",
                    "be4" : "before",
                    "bfn" : "bye for now",
                    "blvd" : "boulevard",
                    "bout" : "about",
                    "brb" : "be right back",
                    "bros" : "brothers",
                    "brt" : "be right there",
                    "bsaaw" : "big smile and a wink",
                    "btw" : "by the way",
                    "bwl" : "bursting with laughter",
                    "c/o" : "care of",
                    "cet" : "central european time",
                    "cf" : "compare",
                    "cia" : "central intelligence agency",
                    "csl" : "can not stop laughing",
                    "cu" : "see you",
                    "cul8r" : "see you later",
                    "cv" : "curriculum vitae",
                    "cwot" : "complete waste of time",
                    "cya" : "see you",
                    "cyt" : "see you tomorrow",
                    "dae" : "does anyone else",
                    "dbmib" : "do not bother me i am busy",
                    "diy" : "do it yourself",
                    "dm" : "direct message",
                    "dwh" : "during work hours",
                    "e123" : "easy as one two three",
                    "eet" : "eastern european time",
                    "eg" : "example",
                    "embm" : "early morning business meeting",
                    "encl" : "enclosed",
                    "encl." : "enclosed",
                    "etc" : "and so on",
                    "faq" : "frequently asked questions",
                    "fawc" : "for anyone who cares",
                    "fb" : "facebook",
                    "fc" : "fingers crossed",
                    "fig" : "figure",
                    "fimh" : "forever in my heart", 
                    "ft." : "feet",
                    "ft" : "featuring",
                    "ftl" : "for the loss",
                    "ftw" : "for the win",
                    "fwiw" : "for what it is worth",
                    "fyi" : "for your information",
                    "g9" : "genius",
                    "gahoy" : "get a hold of yourself",
                    "gal" : "get a life",
                    "gcse" : "general certificate of secondary education",
                    "gfn" : "gone for now",
                    "gg" : "good game",
                    "gl" : "good luck",
                    "glhf" : "good luck have fun",
                    "gmt" : "greenwich mean time",
                    "gmta" : "great minds think alike",
                    "gn" : "good night",
                    "g.o.a.t" : "greatest of all time",
                    "goat" : "greatest of all time",
                    "goi" : "get over it",
                    "gps" : "global positioning system",
                    "gr8" : "great",
                    "gratz" : "congratulations",
                    "gyal" : "girl",
                    "h&c" : "hot and cold",
                    "hp" : "horsepower",
                    "hr" : "hour",
                    "hrh" : "his royal highness",
                    "ht" : "height",
                    "ibrb" : "i will be right back",
                    "ic" : "i see",
                    "icq" : "i seek you",
                    "icymi" : "in case you missed it",
                    "idc" : "i do not care",
                    "idgadf" : "i do not give a damn fuck",
                    "idgaf" : "i do not give a fuck",
                    "idk" : "i do not know",
                    "ie" : "that is",
                    "i.e" : "that is",
                    "ifyp" : "i feel your pain",
                    "IG" : "instagram",
                    "iirc" : "if i remember correctly",
                    "ilu" : "i love you",
                    "ily" : "i love you",
                    "imho" : "in my humble opinion",
                    "imo" : "in my opinion",
                    "imu" : "i miss you",
                    "iow" : "in other words",
                    "irl" : "in real life",
                    "j4f" : "just for fun",
                    "jic" : "just in case",
                    "jk" : "just kidding",
                    "jsyk" : "just so you know",
                    "l8r" : "later",
                    "lb" : "pound",
                    "lbs" : "pounds",
                    "ldr" : "long distance relationship",
                    "lmao" : "laugh my ass off",
                    "lmfao" : "laugh my fucking ass off",
                    "lol" : "laughing out loud",
                    "ltd" : "limited",
                    "ltns" : "long time no see",
                    "m8" : "mate",
                    "mf" : "motherfucker",
                    "mfs" : "motherfuckers",
                    "mfw" : "my face when",
                    "mofo" : "motherfucker",
                    "mph" : "miles per hour",
                    "mr" : "mister",
                    "mrw" : "my reaction when",
                    "ms" : "miss",
                    "mte" : "my thoughts exactly",
                    "nagi" : "not a good idea",
                    "nbc" : "national broadcasting company",
                    "nbd" : "not big deal",
                    "nfs" : "not for sale",
                    "ngl" : "not going to lie",
                    "nhs" : "national health service",
                    "nrn" : "no reply necessary",
                    "nsfl" : "not safe for life",
                    "nsfw" : "not safe for work",
                    "nth" : "nice to have",
                    "nvr" : "never",
                    "nyc" : "new york city",
                    "oc" : "original content",
                    "og" : "original",
                    "ohp" : "overhead projector",
                    "oic" : "oh i see",
                    "omdb" : "over my dead body",
                    "omg" : "oh my god",
                    "omw" : "on my way",
                    "p.a" : "per annum",
                    "p.m" : "after midday",
                    "pm" : "prime minister",
                    "poc" : "people of color",
                    "pov" : "point of view",
                    "pp" : "pages",
                    "ppl" : "people",
                    "prw" : "parents are watching",
                    "ps" : "postscript",
                    "pt" : "point",
                    "ptb" : "please text back",
                    "pto" : "please turn over",
                    "qpsa" : "what happens", #"que pasa",
                    "ratchet" : "rude",
                    "rbtl" : "read between the lines",
                    "rlrt" : "real life retweet", 
                    "rofl" : "rolling on the floor laughing",
                    "roflol" : "rolling on the floor laughing out loud",
                    "rotflmao" : "rolling on the floor laughing my ass off",
                    "rt" : "retweet",
                    "ruok" : "are you ok",
                    "sfw" : "safe for work",
                    "sk8" : "skate",
                    "smh" : "shake my head",
                    "sq" : "square",
                    "srsly" : "seriously", 
                    "ssdd" : "same stuff different day",
                    "tbh" : "to be honest",
                    "tbs" : "tablespooful",
                    "tbsp" : "tablespooful",
                    "tfw" : "that feeling when",
                    "thks" : "thank you",
                    "tho" : "though",
                    "thx" : "thank you",
                    "tia" : "thanks in advance",
                    "til" : "today i learned",
                    "tl;dr" : "too long i did not read",
                    "tldr" : "too long i did not read",
                    "tmb" : "tweet me back",
                    "tntl" : "trying not to laugh",
                    "ttyl" : "talk to you later",
                    "u.s." : 'usa',
                    "u" : "you",
                    "u2" : "you too",
                    "u4e" : "yours for ever",
                    "utc" : "coordinated universal time",
                    "w/" : "with",
                    "w/o" : "without",
                    "w8" : "wait",
                    "wassup" : "what is up",
                    "wb" : "welcome back",
                    "wtf" : "what the fuck",
                    "wtg" : "way to go",
                    "wtpa" : "where the party at",
                    "wuf" : "where are you from",
                    "wuzup" : "what is up",
                    "wywh" : "wish you were here",
                    "yd" : "yard",
                    "ygtr" : "you got that right",
                    "ynk" : "you never know",
                    "zzz" : "sleeping bored and tired"
                    }
sample_abbr_pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) \
                            for key in sample_abbr.keys()) + r')(?!\w)')

def find_all_URL(text):
    match_obj = re.findall(https_pattern, text)
    matches['URL'] += len(match_obj)

def remove_URL(text):
    # Remove URL's from the string text:
    text = re.sub(https_pattern, "", text)
    return text

def find_all_html(text):
    match_obj = re.findall(html_pattern, text)
    matches['html'] += len(match_obj)

def remove_html(text):
    # Remove html from the string text:
    text = re.sub(html_pattern, "", text)
    return text

def find_all_spec_char(text):
    match_obj = re.findall(emoji_pattern, text)
    matches['spec_char'] += len(match_obj)

def remove_spec_char(text):
    # Remove special characters (like emojis and different graphic characters):
    text = emoji_pattern.sub(r'', text)
    return text

def find_all_at(text):
    match_obj = at_pattern.findall(text)
    matches['at'] += len(match_obj)

def remove_at(text):
    text = at_pattern.sub(r'', text)
    return text

def find_all_hashtags(text):
    match_obj = hashtag_pattern.findall(text)
    matches['hashtags'] += len(match_obj)

def remove_hashtags(text):
    # Remove hashtags from the string text:
    text = hashtag_pattern.sub(r'\1', text)
    return text

def find_all_non_ascii(text):
    match_obj = non_ascii_pattern.findall(text)
    matches['non_ascii'] += len(match_obj)

def remove_non_ascii(text):
    # Remove non-ASCII characters from the string text
    text = non_ascii_pattern.sub(r'', text)
    return text

def find_all_multiples(text):
    match_obj_1 = multiples_pattern_1.findall(text)
    matches['multiples_!'] += len(match_obj_1)
    
    match_obj_2 = multiples_pattern_2.findall(text)
    matches['multiples_?'] += len(match_obj_2)

def remove_multiples(text):
    text = multiples_pattern_1.sub(r'!', text)
    text = multiples_pattern_2.sub(r'?', text)
    return text

def remove_punct(text):
    # Remove all punctuation characters except dot '.':
    return re.sub(r'[]!"$%&\'()*+,/:;=#@?[\\^_`{|}~-]+', "", text)

def find_all_the_rest(text):
    modif_obj = sample_typos_slang_pattern.findall(text)
    matches['typos'] += len(modif_obj)
    
    modif_obj = sample_acronyms_pattern.findall(text)
    matches['acronyms'] += len(modif_obj)
    
    modif_obj = sample_abbr_pattern.findall(text)
    matches['abbr'] += len(modif_obj)
    
def remove_the_rest(text): 
    text = sample_typos_slang_pattern.sub(lambda x: sample_typos_slang[x.group()], text)
    text = sample_acronyms_pattern.sub(lambda x: sample_acronyms[x.group()], text)
    text = sample_abbr_pattern.sub(lambda x: sample_abbr[x.group()], text)
    return text
  
def determine_quantities(text):
    find_all_URL(text)
    find_all_html(text)
    find_all_spec_char(text)
    find_all_at(text)
    find_all_hashtags(text)
    find_all_non_ascii(text)
    find_all_multiples(text)
    find_all_the_rest(text)
        
def process_text(text):
    text = remove_URL(text)
    text = remove_html(text)
    text = remove_spec_char(text)
    text = remove_at(text)  # At symbols (@) with following ater it alphanumerical characters
    text = remove_hashtags(text)
    text = remove_non_ascii(text)
    text = remove_multiples(text) # Remove repeated characters '!' and '?'
#     text = remove_punct(text)
    text = text.lower()
    text = remove_the_rest(text)
    text = contractions.fix(text) # Remove english contractions with the 'contractions' module
    text = text.strip()
    return text

In [ ]:
# for key in matches.keys():
#     matches[key] = 0

# modified_text = process_text("BREAKING: Fairfax County firefighter placed on admin leave amid probe into Facebook post about putting police in 'body bags' dept. says.")
# print(modified_text)
# for key in matches.keys():
#     print(f'В тестовом выражении найдено {key}\'с: {matches[key]}.')

#### We calculate now the quantities of all kinds of characteristic symbols in all the given tweets.

In [ ]:
for key in matches.keys():
    matches[key] = 0

train_tw.text.apply(determine_quantities)

print('В твиттах найдено:')
for key in matches.keys():
    print(f'{key}: {matches[key]}')

#### Let us remove all or some of them.

In [7]:
train_tw['processed_text'] = train_tw.text.apply(process_text)

In [8]:
for row in range(1017, 1042):
    print('Original tweet:')
    print (train_tw.loc[row, 'text'])
    print('-'*50)
    print('Cleaned tweet:')
    print(train_tw.loc[row, 'processed_text'])
    print('*'*50)

Original tweet:
@SlikRickDaRula Drake really body bagging peeps man ?? he really bout it
--------------------------------------------------
Cleaned tweet:
drake really body bagging peeps man ? he really about it
**************************************************
Original tweet:
@FoxNews @JenGriffinFNC When you call to report dangerous activity tell em to have body bags on arrival.
--------------------------------------------------
Cleaned tweet:
when you call to report dangerous activity tell them to have body bags on arrival.
**************************************************
Original tweet:
Micom 2015 Summer Contrast Candy Color Bowknot Cross Body Tote Shoulder Bags for Womengirls with Micom Zip Po http://t.co/sQMTKKJiMJ
--------------------------------------------------
Cleaned tweet:
micom 2015 summer contrast candy color bowknot cross body tote shoulder bags for womengirls with micom zip po
**************************************************
Original tweet:
HOBO Hobo Vintage Shira 

#### Transforming the type of all the cells in the columns 'keyword' and 'location' to str.

In [9]:
train_tw.keyword = train_tw['keyword'].astype('str')
train_tw.location = train_tw['location'].astype('str')

#### Let's remove rows with empty cells. And in one case, we will try to remove from the data observations without a specified keyword. In another cas, we will try to leave them.

In [10]:
train_tw = train_tw[train_tw['keyword'] != 'nan']
train_tw.dropna(subset='target')
train_tw.dropna(subset='text')

,id,keyword,location,text,target,processed_text
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,wholesale markets ablaze
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,we always try to bring the heavy. metal retweet
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,africanbaze: breaking news:nigeria flag set ab...
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,crying out for more! set me ablaze
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,on plus side look at the sky last night it was...
...,...,...,...,...,...,...
7578,10830,wrecked,nan,@jt_ruff23 @cameronhacker and I wrecked you both,0,and i wrecked you both
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0,three days off from work and they have pretty ...
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0,fx forex trading cramer: iger's 3 words that w...
7581,10833,wrecked,Lincoln,@engineshed Great atmosphere at the British Li...,0,great atmosphere at the british lion gig tonig...


In [11]:
train_tw.shape

(7552, 6)

#### Let's see now tweets' lengths after their processing.

In [ ]:
train_tw['new_tweet_length'] = [len(tweet.split()) for tweet in train_tw.processed_text]
print(train_tw['new_tweet_length'].quantile([0.01, 0.05, 0.10, 0.5, 0.9, 0.95, 0.99]))

In [ ]:
plot_tweets_length_distrib('processed_text')

In [21]:
train_tw.isna().any()

id                           False
keyword                      False
location                     False
text                         False
target                       False
processed_text               False
FastText vectors of words    False
dtype: bool

In [13]:
train_tw.reset_index(drop=True, inplace=True)

In [14]:
train_tw.head()

,id,keyword,location,text,target,processed_text
0,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,wholesale markets ablaze
1,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,we always try to bring the heavy. metal retweet
2,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,africanbaze: breaking news:nigeria flag set ab...
3,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,crying out for more! set me ablaze
4,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,on plus side look at the sky last night it was...


#### We add FastText vectors for words in the tweets.

In [ ]:
import fasttext.util

fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 (30.56%) [===============>                                   ]==============>                                    ]==============>                                    ]==============>                                    ]==============>                                    ]==============>                                    ]==============>                                    ]==============>                                    ]==============>                                    ]==============>                                    ]==============>                                    ]==============>                                    ]==============>                                    ]==============>                                    ]==============>                                    ]==============>                                    ]==============>                                    ]==============>                                    ]==============>                                    ]=

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
ft.get_word_vector('he')

In [16]:
def get_word_vectors(text):
    vectors = {}
    words_list = text.split(' ')
    for word in words_list:
        vectors[word] = ft.get_word_vector(word)
    return vectors

Let's add to the data a column with FastText vectrors for every word.

In [17]:
train_tw['FastText vectors of words'] = train_tw.processed_text.apply(get_word_vectors)

In [20]:
train_tw.loc[7546, 'FastText vectors of words']

{'...': array([ 0.03674697,  0.00247653, -0.1421954 , -0.06621412, -0.18753849,
         0.09782217, -0.1612324 ,  0.01616048, -0.0318169 ,  0.02479623,
        -0.14685988, -0.06056542,  0.03590964,  0.01742998,  0.07599189,
        -0.02099836, -0.10989551, -0.13091148, -0.02892713, -0.01132542,
         0.00947135,  0.02354666, -0.01896219, -0.04704021, -0.07059688,
         0.02300329, -0.01691282,  0.00094182,  0.03244984,  0.08053891,
        -0.04134432,  0.06921591,  0.01079007, -0.03721829, -0.00768988,
         0.11532577,  0.05966695, -0.02092543,  0.00906137, -0.05680125,
         0.03183767, -0.12626505, -0.05363214, -0.11006853,  0.16123791,
         0.09676662, -0.06532615,  0.06740695, -0.01349872, -0.09433304,
         0.06780944, -0.01364387,  0.1669063 ,  0.00524048, -0.04239861,
         0.09099621,  0.02101782,  0.07586063, -0.0589465 ,  0.10534996,
         0.06651051,  0.03767153, -0.00898773,  0.02131439,  0.0564755 ,
         0.02713151, -0.09275679,  0.0057273

Let's save this table and download it later for models.

In [22]:
train_tw.to_pickle('Cleaned_data_with_fasttext_vectors.pkl',
                   compression={'method': 'gzip'})

In [ ]:
train_tw.to_csv('Cleaned_data_with_fasttext_vectors.csv')

### Evaluation of classifiers' results.

In [23]:
def plot_conf_matrix(y_test, y_preds, save=False):
    
    cm = confusion_matrix(y_test, y_preds)
    fig, ax = plt.subplots()
    tick_marks = np.as_array([0, 1])
    plt.xticks(tick_marks)
    plt.yticks(tick_marks)
    # create heatmap
    sns.heatmap(pd.DataFrame(cm), annot=True, cmap="BuPu" ,fmt='g')
    ax.xaxis.set_label_position("top")
    plt.tight_layout()
    plt.title('Confusion matrix', y=1.1)
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    
    if save==True:
        save_plot(fig)
                
    plt.show()

    
def plot_roc_auc(y_test, y_preds_proba, save=False):

    model_auc = roc_auc_score(y_test, y_preds_proba, multi_class='ovr', average='weighted')
    print(f'ROC AUC={model_auc:.4f}')
    # Let's compute ROC AUC
    fpr = {} # False Positive Rate
    tpr = {} # True Positive Rate
    thresh ={}
    n_class = y_preds_proba.shape[1] # 2 classes: disaster tweets and calm ones

    colors = ['tab:blue', 'tab:orange']
    plt.figure(figsize=(8, 6))
    for i in range(n_class):    
        fpr[i], tpr[i], thresh[i] = roc_curve(y_test, y_preds_proba[:,i], pos_label=i)
        plt.plot(fpr[i], tpr[i], linestyle='--', color=colors[i],
                 label=f'{i} vs Rest (area = {auc(fpr[i], tpr[i]):0.2f})')
        
    plt.plot([0, 1], [0, 1], color='tab:gray', linestyle='--')
    plt.title('ROC curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive rate')
    plt.legend(loc='best')
    
    if save==True:
        save_plot(fig)
        
    plt.show()

def save_plot(fig):
    now_time = strftime("%Y-%m-%d %H-%M-%S", gmtime())
    current_dir = os.getcwd()
    # Let's create a new folder to save the graph
    new_path = os.path.join(current_dir, 'results')
    os.makedirs(new_path, exist_ok=True)
    file_path = os.path.join(new_path, 'results', f'conf_matrix {now_time}.png')

    fig.savefig(file_path)

In [24]:
def evaluate_results(y_test, y_preds, y_preds_proba, save=False):
    
    plot_conf_matrix(y_test, y_preds, save=save)
    plot_roc_auc(y_test, y_preds_proba, save=save)
        
    print(classification_report(y_test, y_preds, target_names=labels_str2int.keys()))
    print(f'Accuracy: {accuracy_score(y_test, y_preds)}')
    print(f'f1_score: {f1_score(y_test, y_preds, average="weighted")}')

### Training classifiers and classifying test tweets.

#### Splitting data to train and validation sets:

In [30]:
X = train_tw.processed_text
Y = train_tw.target
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42,
                                                    shuffle=True)

In [ ]:
# import gensim.downloader as api

# glove_vectors = api.load('glove-twitter-200')
# print(glove_vectors.get_vector('tweeting'))

In [27]:
def fasttext_vectorizer(row):
    summarized_vector = np.zeros(300)
    vector_dict = train_tw.loc[row, 'FastText vectors of words']
    for vector in vector_dict.values():
        summarized_vector += vector
    number = len(vector_dict.values())
    sentence_vector = summarized_vector / number
    
    return sentence_vector

In [28]:
row = 3467
vector = fasttext_vectorizer(row)
print(vector)

[-2.15249974e-02  2.36509012e-02  2.82061880e-03  3.58113883e-02
 -6.37254663e-02  1.45013386e-02  4.84981358e-02  1.69377693e-03
  4.39762336e-03  1.26166982e-02 -1.94419922e-02 -2.15401646e-02
  1.34540233e-02 -5.45352526e-03 -9.75820143e-03  1.30244776e-01
  9.26620909e-04 -3.90014672e-02 -6.40731557e-02  2.79760465e-03
  1.44303087e-02  7.56230746e-02  2.68316135e-02 -7.62357616e-02
  6.20249112e-02 -9.05607635e-02  2.59574578e-02 -1.01871844e-02
 -2.22188075e-02  1.55884234e-01  2.00616152e-02 -3.20104673e-02
 -2.80992116e-02 -1.14195693e-01 -8.12390354e-05  6.88867469e-03
 -2.00430758e-02  2.24540188e-02 -2.05510654e-02 -3.92397541e-02
  3.48514601e-02  2.40694771e-02 -1.22828281e-03  1.20356215e-01
  5.45112267e-02  6.19119983e-02  3.54249417e-02  1.23900462e-01
 -3.32306103e-02  5.17615757e-02 -5.72483511e-03 -3.56718480e-02
  1.00347520e-02 -6.97752878e-02 -1.93694900e-02  7.02706957e-03
 -5.88082434e-02 -1.98673005e-02 -1.12946393e-01 -6.34167367e-03
 -9.37034951e-03 -1.03971

In [33]:
tfidf_vectorizer = TfidfVectorizer()
x_train = tfidf_vectorizer.fit_transform(x_train)
x_val = tfidf_vectorizer.transform(x_val)

AttributeError: lower not found

In [ ]:
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)
y_preds = lr.predict(X_test)
y_preds_proba = lr.predict_proba(X_test)
evaluate_results(y_test, y_preds, y_preds_proba)